# Modelamiento del sistema

* **Temporalidad:** Discreta (observaciones diarias).

* **Demanda:**
  Sea $D$ la demanda diaria del medicamento.

  $$
  D \sim \text{Poisson}(\lambda = 3)
  $$

  $$
  P(D = k) = \frac{3^k e^{-3}}{k!}, \quad k = 0,1,2,\dots
  $$

  $$
  P(D \geq k) = \sum_{r=k}^{\infty} \frac{3^r e^{-3}}{r!}
  $$

* **Variable de estado:**

  $$
  X_n = \text{Número de unidades del medicamento disponibles al inicio del día } n
  $$

* **Espacio de estados:**

  $$
  S_X = \{0,1,2,3,4,5\}
  $$

* **Colaboración hospitalaria:**
  Si al final del día el inventario queda en **0**, la farmacia entra en cuarentena y el inventario del día siguiente depende de la colaboración de hospitales.

  $$
  Y \sim \text{Binomial}(n=5, p=0.07)
  $$

  $$
  P(Y = k) = \binom{5}{k}(0.07)^k (0.93)^{5-k}, \quad k=0,1,\dots,5
  $$

* **Política de transición:**

$$
p_{ij} =
\begin{cases}
\mathbb{P}(D = i - j), & i \geq 2,\ j \neq 0 \\
\mathbb{P}(D \geq i), & i \geq 2,\ j = 0 \\
\mathbb{P}(D = 5 - j), & 1 \leq i < 2,\ j \neq 0 \\
\mathbb{P}(D \geq 5), & 1 \leq i < 2,\ j = 0 \\
\mathbb{P}(Y = j), & i = 0,\ j \in \{0,1,\dots,5\} \\
0, & \text{d.l.c.}
\end{cases}
$$




In [13]:
import numpy as np
from scipy.stats import poisson, binom
from jmarkov.dtmc import dtmc

In [23]:
# Parametros

lmbda = 3 # Tasa de llegada
n = 5
p = 0.07 # Probabilidad
u = 2 # Umbral de reposicion

In [24]:
# Espacio de estados
estados = np.array([i for i in range(n+1)])

# Matriz de transicion
matriz_transicion = np.zeros((len(estados), len(estados)))


In [25]:
def matriz_transicion_func(matriz, lbd, probabilidad, umbral):
    for i in range(len(estados)):
        for j in range(len(estados)):

            if i >= umbral and j != 0:
                matriz[i, j] = poisson.pmf(i-j, lbd)
            elif i >= umbral and j == 0:
                matriz[i, j] = 1-poisson.cdf(i-1, lbd)
            elif (1 <= i < umbral) and j != 0:
                matriz[i, j] = poisson.pmf(5-j, lbd)
            elif (1 <= i < umbral) and j == 0:
                matriz[i, j] = 1-poisson.cdf(5-1, lbd)
            elif  i == 0 and j in estados:
                matriz[i, j] = binom.pmf(j, 5, probabilidad)
            else:
                matriz[i, j] = 0

matriz_transicion_func(matriz_transicion, lmbda, p, u)
matriz_transicion

array([[6.95688369e-01, 2.61818204e-01, 3.94134930e-02, 2.96660700e-03,
        1.11646500e-04, 1.68070000e-06],
       [1.84736755e-01, 1.68031356e-01, 2.24041808e-01, 2.24041808e-01,
        1.49361205e-01, 4.97870684e-02],
       [8.00851727e-01, 1.49361205e-01, 4.97870684e-02, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00],
       [5.76809919e-01, 2.24041808e-01, 1.49361205e-01, 4.97870684e-02,
        0.00000000e+00, 0.00000000e+00],
       [3.52768111e-01, 2.24041808e-01, 2.24041808e-01, 1.49361205e-01,
        4.97870684e-02, 0.00000000e+00],
       [1.84736755e-01, 1.68031356e-01, 2.24041808e-01, 2.24041808e-01,
        1.49361205e-01, 4.97870684e-02]])

In [26]:
cadena = dtmc(matriz_transicion, estados)

### a. **¿Cuál es la probabilidad de que el inventario sea igual a 5 dentro de 2 días, si hoy hay 1 unidad?**

Queremos calcular:

$$
P(X_2 = 5 \mid X_0 = 1)
$$

Esto corresponde a una **transición de dos pasos**, es decir:

$$
P(X_2 = 5 \mid X_0 = 1) = \big[P^2\big]_{1,5}
$$



In [27]:
alpha = np.zeros(len(estados))
alpha[1] = 1 # estamos en el estado de 1 unidad al 100%

dist_paso2 = cadena.transient_probabilities(n=2, alpha=alpha) # n = 2 pasos
probabilidad = round(dist_paso2[5],4)
print(f"La probabilidad de que el inventario sea igual a 5 dentro de 2 dias, si hoy hay 1 unidad es de {probabilidad}")


La probabilidad de que el inventario sea igual a 5 dentro de 2 dias, si hoy hay 1 unidad es de 0.0108


### b. **¿Qué pasaría si el umbral de reposición cambiara de 2 a 3?**
¿Se reduciría el riesgo de quiebre?

* **Nueva Política de transición:**

$$
p_{ij} =
\begin{cases}
\mathbb{P}(D = i - j), & i \geq 3,\ j \neq 0 \\
\mathbb{P}(D \geq i), & i \geq 3,\ j = 0 \\
\mathbb{P}(D = 5 - j), & 1 \leq i < 3,\ j \neq 0 \\
\mathbb{P}(D \geq 5), & 1 \leq i < 3,\ j = 0 \\
\mathbb{P}(Y = j), & i = 0,\ j \in \{0,1,\dots,5\} \\
0, & \text{d.l.c.}
\end{cases}
$$


In [28]:
matriz_transicion_alt = np.zeros((len(estados), len(estados)))
u_nuevo = 3
matriz_transicion_func(matriz_transicion_alt, lmbda, p, u_nuevo)

matriz_transicion_alt

array([[6.95688369e-01, 2.61818204e-01, 3.94134930e-02, 2.96660700e-03,
        1.11646500e-04, 1.68070000e-06],
       [1.84736755e-01, 1.68031356e-01, 2.24041808e-01, 2.24041808e-01,
        1.49361205e-01, 4.97870684e-02],
       [1.84736755e-01, 1.68031356e-01, 2.24041808e-01, 2.24041808e-01,
        1.49361205e-01, 4.97870684e-02],
       [5.76809919e-01, 2.24041808e-01, 1.49361205e-01, 4.97870684e-02,
        0.00000000e+00, 0.00000000e+00],
       [3.52768111e-01, 2.24041808e-01, 2.24041808e-01, 1.49361205e-01,
        4.97870684e-02, 0.00000000e+00],
       [1.84736755e-01, 1.68031356e-01, 2.24041808e-01, 2.24041808e-01,
        1.49361205e-01, 4.97870684e-02]])

In [29]:
mc_vieja = dtmc(matriz_transicion, estados)
mc_nueva = dtmc(matriz_transicion_alt, estados)

pi_vieja = mc_vieja.steady_state()
pi_nueva = mc_nueva.steady_state()

print(f"Distribucion estacionaria con umbral 2: {np.round(pi_vieja,4)}")
print(f"Distribucion estacionaria con umbral 3: {np.round(pi_nueva,4)}")
print("")
print(f"Probabilidad de quiebre con umbral 2: {round(pi_vieja[0],4)}")
print(f"Probabilidad de quiebre con umbral 3: {round(pi_nueva[0],4)}")

if pi_nueva[0] < pi_vieja[0]:
    print("Se reduce el riesgo de quiebre")
else:
    print("No se reduce el riesgo de quiebre")

Distribucion estacionaria con umbral 2: [0.5648 0.2248 0.098  0.0634 0.0373 0.0118]
Distribucion estacionaria con umbral 3: [0.4759 0.2214 0.1289 0.0975 0.058  0.0184]

Probabilidad de quiebre con umbral 2: 0.5648
Probabilidad de quiebre con umbral 3: 0.4759
Se reduce el riesgo de quiebre


### c. **¿Cuánto impacta la probabilidad de colaboración hospitalaria ($p = 0.07$) en la confiabilidad del sistema?**
¿Cómo cambiarían las probabilidades si esta subiera a $p = 0.2$?

In [31]:
p_nuevo = 0.2 # Nueva probabilidad de colaboracion

matriz_transicion_colab = np.zeros((len(estados), len(estados))) # Matriz de transicion con nueva probabilidad

matriz_transicion_func(matriz_transicion_colab, lmbda, p_nuevo, u)

mc_colab = dtmc(matriz_transicion_colab, estados) # Cadena de Markov con nueva probabilidad

alpha_cuarentena = np.zeros(len(estados)) # Distribucion inicial
alpha_cuarentena[0] = 1 # Estamos en el estado de quiebre al 100%
dist_colab = mc_colab.transient_probabilities(n=1, alpha=alpha_cuarentena) # Distribucion despues de 1 dia

# Probabilidad de tener > 1 unidad despues de 1 dia
probabilidad_colab = round(sum(dist_colab[1:]),4)

dist_vieja = mc_vieja.transient_probabilities(n=1, alpha=alpha_cuarentena)

# Probabilidad de tener > 1 unidad despues de 1 dia con probabilidad vieja
probabilidad_vieja = round(sum(dist_vieja[1:]),4)

print(f"Probabilidad de tener mas de 1 unidad despues de 1 dia con p=0.07: {probabilidad_vieja}")
print(f"Probabilidad de tener mas de 1 unidad despues de 1 dia con p=0.2: {probabilidad_colab}")



Probabilidad de tener mas de 1 unidad despues de 1 dia con p=0.07: 0.3043
Probabilidad de tener mas de 1 unidad despues de 1 dia con p=0.2: 0.6723
